# Introduction to TD and Q-Learning 

## CSCI E-82A

## Stephen Elston


In the previous lesson we explored Monte Carlo **reinforcement learning**. MC RL required that the returns for an entire episode be computed before any values are available for use. The advantage of this approach is the the full set of returns are used for state value or action value estimates. But, how can we get state values or action-values in fewer time steps? It turns out there are algorithms which can compute these estimates in as few as one step known as **time difference learning** or **TD learning** and **Q-learning**. 

Recall that reinforcement learning has several distinctive characteristics, which differentiate this method from other machine learning and dynamic programming:
- **No Markov model** needs to be specified for reinforcement learning, in contrast to dynamic programming.
- Like dynamic programming, reinforcement learning **optimizes a reward function**. This is in contrast to supervised and unsupervised learning which use an error or objective function.  
- Reinforcement learning algorithms learn by **experience**. Over time, the algorithm learns a model of the environment and these results are used to optimize the expected reward. Learning from experience is in contrast to supervised learning which uses known marked cases. 
- Reinforcement learning agents take **actions** and only receive **state** and **rewards** from the environment. These are the only interaction between the RL agent and the environment.    

The interaction between a reinforcement learning agent and the environment are illustrated in the figure below. Notice that the only feedback the agent receives from the environment is reward and state.   

<img src="img/RL_AgentModel.JPG" alt="Drawing" style="width:500px; height:300px"/>
<center> **Reinforcement Learning Agent and Environment** </center>  

The ability to learn from experience is an attractive concept. This method of learning seems to mimic human learning. However, reinforcement learning has proven difficult to use in real-world applications. For a review of successes and problems arising when applying RL to robotics see [Kobler et. al.](https://www.ias.informatik.tu-darmstadt.de/uploads/Publications/Kober_IJRR_2013.pdf). At the present time, RL has mostly succeeded in cases where simulations can be used to gain experience. 

**Suggested readings** for TD and Q reinforcement learning, Chapters 6 and 7 of Sutton and Barto, second edition, provides a good introductions, including many alternative algorithms and details not discussed here.   

## TD Prediction Model

In a previous lesson we examined a general update model:

$$NewValue = OldValue + LearningRate * ErrorTerm$$

Following this general formulation the update for MC RL can be written as:

$$V(S_t) = V(S_t) + \alpha \big[ G_T - V(S_t) \big]$$

Here,   
$\alpha = $ the learning rate,    
$\big[ G_T - V(S_t) \big] = $ the error term is the difference between the return $G_t$ and the value, $V(S_t)$.   

   
Using the same general formulation we can create and single time step update model know as the **one step time difference** or **TD(0)** algorithm.

$$V(S_t) = V(S_t) + \alpha \big[ R_{t+1} + V(S_{t+1}) - V(S_t) \big]$$

Where,  
$\delta_t =  R_{t+1} + V(S_{t+1}) - V(S_t) = $ the **TD error**,  
$R_{t+1} = $ the return for the next time step,   
$V(S_t) = $ is the value at time step t.   

Like dynamic programming algorithms, the TD algorithm **bootstraps**. The return and estimated value at the next time step, $V(S_{t+1})$, are from previous samples. However, unlike MC RL which does not bootstrap, the TD(0) algorithm produces an estimate of $V(S_t)$ in only one time step, rather than waiting to reach the terminal state at the end of the episode. This fact can be seen by examining the backup diagram shown below:

<img src="img/TD0.JPG" alt="Drawing" style="width:60px; height:150px"/>
<center> **Backup Diagram of TD(0)** </center>

## Example of Time Difference RL

With this short introduction TD RL in mind, let's try an example. We will sample the value function using a basic TD(0) algorithm here. 

**Navigation** to a goal is a significant problem in robotics. Real-world navigation is rather complex. Therefore, we will continue to use a simple analog called a **grid world**. The grid world for this problem should be familiar by now, and is shown below. 

<img src="img/GridWorld.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **A 4x4 Grid World with Terminal State** </center>

The grid world consists of a 4x4 set of positions the robot can occupy. Each position is considered a state. The goal is to navigate to state 0, the goal, in the minimum steps. We will explore methods to find policies which reach this goal and achieve maximum reward. 

Grid position 0 is the goal and a **terminal state**. There are no possible state transitions out of this position. The presence of a terminal state makes this an **episodic Markov random process**. For each episode sampled the robot can start in any other random position, $\{ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 \}$. This random selection process makes this a **random start** Monte Carlo algorithm. The episode terminates when the robot enters the terminal position (state 0).  

### Representation For TD RL

In each state, there are four possible actions the robot can take:
- up, u
- down, d,
- left, l
- right, r

The TD RL agent has no model. Therefore, beyond these allowed actions and a list of states visited, all other information is encapsulated in the environment and is unobservable by the agent. This is the key difference between reinforcement learning and dynamic programming. 

The RL agent must **explore** to find the possible actions when it is in some particular state. To simplify our example, we encode, or represent, these possibilities in a dictionary as shown in the code block below. The keys of the inner dictionary are the possible actions and the values are the **successor state**, $s'$, for that transition.  

As with MC RL, we can start with an arbitrary initial policy for TD RL. The TD RL agent will then improve this policy and in the process will **learn the Markov process model**. 

We need to define the transition probabilities for the initial policy. We set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. As there are 4 possible transitions from each state, this means all transition probabilities are 0.25. In other words, this is a random policy which does not favor any particular plan. 

The initial uniform transition probabilities are encoded using a dictionary of dictionaries. The organization of this data structure is identical to the foregoing data structure. 

The robot receives the following rewards:
- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the TD(0) RL agent**. The agent must **learn** the rewards by sampling the environment. Here the rewards are in the form of action values. We encode these rewards in the same type of dictionary structure used for the foregoing structures. 

In [ ]:
## import numpy for latter
import numpy as np
import numpy.random as nr

## Define the transition dictonary of dictionaries:
neighbors = {0:{'u':0, 'd':0, 'l':0, 'r':0},
          1:{'u':1, 'd':5, 'l':0, 'r':2},
          2:{'u':2, 'd':6, 'l':1, 'r':3},
          3:{'u':3, 'd':7, 'l':2, 'r':3},
          4:{'u':0, 'd':8, 'l':4, 'r':5},
          5:{'u':1, 'd':9, 'l':4, 'r':6},
          6:{'u':2, 'd':10, 'l':5, 'r':7},
          7:{'u':3, 'd':11, 'l':6, 'r':7},
          8:{'u':4, 'd':12, 'l':8, 'r':9},
          9:{'u':5, 'd':13, 'l':8, 'r':10},
          10:{'u':6, 'd':14, 'l':9, 'r':11},
          11:{'u':7, 'd':15, 'l':10, 'r':11},
          12:{'u':8, 'd':12, 'l':12, 'r':13},
          13:{'u':9, 'd':13, 'l':12, 'r':14},
          14:{'u':10, 'd':14, 'l':13, 'r':15},
          15:{'u':11, 'd':15, 'l':14, 'r':15}}

policy = {0:{'u':0.25, 'd':0.0, 'l':0.0, 'r':0.0},
                        1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}, 
                        2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}}

rewards = {0:{'u':0.0, 'd':0.0, 'l':0.0, 'r':0.0},
          1:{'u':-1, 'd':-0.1, 'l':10.0, 'r':-0.1},
          2:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          4:{'u':10.0, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          5:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          6:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          7:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          8:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          9:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          10:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          11:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          12:{'u':-0.1, 'd':-1.0, 'l':-1.0, 'r':-0.1},
          13:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          14:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          15:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-1.0}}

### Environment

RL agents sample state values and state action values. The RL agent does not know how many states there are and which states are terminal. The RL agent learns a policy as it proceeds with the learning process. The probabilities of a particular state transition are determined by the current policy. 

The code in the cell below simulates the environment. Given the current state and an action the function returns the next state and reward. Thus, this function simulates the interaction the agent will have with the environment. 

Execute this code and examine the results of the simple test. 

In [ ]:
def state_values(s, action, neighbors = neighbors, rewards = rewards):
    """
    Function simulates the environment
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward = rewards[s][action]
    return (s_prime,reward)

## Test the function
for a in ['u', 'd', 'r', 'l']:
    print(state_values(2, a))

### TD(0) Policy Evaluation

We have everything in place to perform TD(0) policy evaluation for the grid world. The code in the cell below implements the TD(0) algorithm and applies it to the policy in the grid world. Notice that the algorithm makes calls to the aforementioned `state_values` function to find information on successor state and rewards for a transition given a current state and action. Execute this code and examine the results. 

In [ ]:
def td_0_state_values(policy, n_samps, goal, alpha = 0.2, gamma = 0.9):
    """
    Function for TD(0) policy 
    """
    ## Initialize the state list and state values
    states = list(policy.keys())
    v = [0]*len(list(policy.keys()))
    action_index = list(range(len(list(policy[0].keys()))))
    for _ in range(n_samps):
        s = nr.choice(states, size =1)[0]
        a = list(policy[s].keys())[nr.choice(action_index, size = 1)[0]]
        transistion = state_values(s, a)
        v[s] = v[s] + alpha * (transistion[1] +  gamma * v[transistion[0]] - v[s])
    return(v)
    
nr.seed(345)    
np.round(np.array(td_0_state_values(policy, n_samps = 100000, goal = 0)).reshape((4,4)), 4)    

## On Policy vs. Off Policy Algorithms

In this lesson we will explore examples of two broad categories of RL algorithms known as **on policy** and **off policy** methods. 

On policy methods evaluate and improve a single policy. On policy methods converge quickly and often to good solution. In general, **exploration** is performed using $\epsilon$-greedy methods. The TD(0) and MC algorithms we have examined are examples of on policy methods. On policy algorithms are known to have good convergence properties. 

In contrast, off policy methods use two policies. The policy the agent is following is called the **behavior policy**, denoted $b(A_t | S_t)$. The policy being improved is known as the **target policy**, denoted $\pi (A_t | S_t)$. The agent obtains samples of the environment while following the behavior policy. These samples are used to improve the target policy. An advantage of off policy methods is that a deterministic behavior policy can be used while a better target policy is developed. 

## One Step SARSA Algorithm

Now that we have examined the one step TD(0) algorithm for policy (value) evaluation, we need to define a one step algorithm for **control** or **policy improvement**.     

The **state action reward state action** or **SARSA** algorithm is an **on policy** method which uses time differencing to evaluate action values. As you might imagine from the name, this algorithm starts with an **on policy** action from the current state which results in a state transition and a reward. The backup diagram for one step SARSA (SARSA(0)) is shown in the figure below. 

<img src="img/SARSA.JPG" alt="Drawing" style="width:75px; height:150px"/>
<center> **Backup Diagram for SARSA(0)** </center>

Compare the backup diagram for SARSA(0) to the one for TD(0) notice that the order of state and action are reversed between the two algorithms. This realization is a good way to understand the difference. 

The update equation for SARSA(0) is as follows:

$$Q(S_t,A_t) = Q(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q(S_{t+1},A_{t+1}) - Q(S_t,A_t) \big]$$  

Where,   
$\delta_t = R_{t+1} + \gamma Q(S_{t+1},A_{t+1}) - Q(S_t,A_t) = $ the error term,   
$Q(S_t,A_t) = $ is the action value in state S given action A,  
$R_{t+1} = $ is the reward for the next time step,   
$\alpha = $ the learning rate,   
$\gamma = $ discount factor.  

### SARSA(0) Example

The code in the cell below implements the SARSA(0) algorithm to compute the action values for the Grid world given an policy. As with the policy evaluation code, this code makes calls to the environment t find successor states and rewards given actions. The function `select_a_prime` queries the environment to receive the successor state and reward given the current state and action. The successor action is computed following the policy. Additional details on this algorithm can be seen by reading the code comments.  

Execute this code to compute and print the action values for the random walk policy.     

In [ ]:
import copy

def select_a_prime(s_prime, policy, action_index, greedy, goal):
    ## Randomly select an action prime 
    ## Make sure to handle the terminal state
    if(s_prime != goal and greedy): 
        probs = list(policy[s_prime].values())
        a_prime_index = nr.choice(action_index, size = 1, p = probs)[0]
        a_prime = list(policy[s_prime].keys())[a_prime_index]
    else: ## Don't probability weight for terminal state or non-greedy selecttion
        a_prime_index = nr.choice(action_index, size = 1)[0]
        a_prime = list(policy[s_prime].keys())[a_prime_index]   
    return(a_prime_index, a_prime)


def SARSA_0(policy, episodes, goal, alpha = 0.2, gamma = 0.9, epsilon = 0.1):
    """
    Function to perform SARSA(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    
    ## Initialize possible actions and the action values
    action_index = list(range(len(list(policy[0].keys()))))
    Q = np.zeros((len(action_index),len(states)))
    
    current_policy = copy.deepcopy(policy)
    
    for _ in range(episodes): # Loop over the episodes
        ## sample a state at random ensuring it is not terminal state
        s = nr.choice(states, size = 1)[0]
        while(s == goal): s = nr.choice(states, size = 1)[0]
        ## Now choose action given policy
        a_index, a = select_a_prime(s, current_policy, action_index, True, goal)
        
        s_prime = float('inf') # Value of s_prime to start loop
        while(s_prime != goal): # Episode ends where get to terminal state 
            ## The next state given the action
            s_prime, reward = state_values(s, a)
            a_prime_index, a_prime = select_a_prime(s_prime, current_policy, action_index, True, goal)
     
            ## Update the action values
            Q[a_index,s] = Q[a_index,s] + alpha * (reward + gamma * Q[a_prime_index,s_prime] - Q[a_index,s])
            
            ## Set action and state for next iteration
            a = a_prime
            a_index = a_prime_index
            s = s_prime

    return(Q)

Q = SARSA_0(policy, 1000, goal = 0, alpha = 0.2, epsilon = 0.1)

for i in range(4):
    print(np.round(Q[i,:].reshape((4,4)), 4))

The arrays printed above display the action values for each state. The four arrays represent the four possible actions, up, down, left and right. 

### GPI with SARSA(0)

The code in the cell below performs **general policy improvement (GPI)** using SARSA(0) to evaluate action values. Several cycles of SARSA(0) and policy improvement are performed with this code. Notice that policy improvement is $\epsilon$-greedy. The probability of any state transition will never go to zero, allowing continued exploration. Additional details on this algorithm can be seen by reading the code comments.  

Execute this code and examine the results.  

In [ ]:
def SARSA_0_GPI(policy, cycles, episodes, goal, alpha = 0.1, gamma = 0.1, epsilon = 0.1):
    ## iterate over GPI cycles
    current_policy = copy.deepcopy(policy)
    for _ in range(cycles):
        ## Evaluate policy with SARSA
        Q = SARSA_0(policy, episodes = episodes, goal = goal, alpha = alpha, epsilon = epsilon)
        
        for s in list(current_policy.keys()): # iterate over all states
            ## Find the index action with the largest Q values 
            ## May be more than one. 
            max_index = np.where(Q[:,s] == max(Q[:,s]))[0]
            
            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(Q.shape[0])
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder   
                    
    return(current_policy)                    
 

SARSA_0_Policy = SARSA_0_GPI(policy, cycles = 10, episodes = 100, goal = 0, alpha = 0.2, epsilon = 0.01)
SARSA_0_Policy

In [ ]:
np.round(np.array(td_0_state_values(SARSA_0_Policy, n_samps = 10000, goal = 0)).reshape((4,4)),4)

## Q-Learning

As we have just seen, the SARSA algorithm is an on policy action value TD estimation method. The **Q-learning** algorithm is a **off policy** TD action value estimation method. 

The update formula for single step Q-learning or **Q-learning(0)** is:

$$Q(S_t,A_t) = Q(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma max_a Q(S_{t+1},a) - Q(S_t,A_t) \big]$$  

Where,   
$\delta_t = R_{t+1} + \gamma max_a Q(S_{t+1},a) - Q(S_t,A_t) = $ the TD error,   
$max_a = $ the maximum operator applied to all possible actions in state $S_{t+1}$,   
$Q(S_t,A_t) = $ is the action value in state S given action A,  
$R_{t+1} = $ is the reward for the next time step,   
$\alpha = $ the learning rate,   
$\gamma = $ discount factor.  

The use of the operator $max_a$ makes Q-learning greedy. But, why does using this operator result in an off-policy algorithm? To answer this question, examine the backup diagram shown below. 

<img src="img/Q-Learning.JPG" alt="Drawing" style="width:200px; height:150px"/>
<center> **Backup Diagram for one-step Q-Learning** </center>

The $max_a$ greedily picks the action with the greatest value, regardless of policy. Therefore, Q-learning is an off-policy algorithm. 

### Q-Learning Example

The code in the cell below implements the one step Q-learning(0) algorithm. The code is nearly identical to the SARSA(0) code shown previously. The main difference is the addition of the $max_a$ operation when computing the TD error, $\delta_t$.  Additional details on this algorithm can be seen by reading the code comments.  
 
Execute this code for the random walk policy on the grid world and examine the results. 

In [ ]:
def Q_learning_0(policy, neighbors, rewards, episodes, goal, alpha = 0.2, gamma = 0.9):
    """
    Function to perform Q-learning(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    
    ## Initialize possible actions and the action values
    possible_actions = list(rewards[0].keys())
    action_index = list(range(len(list(policy[0].keys()))))
    Q = np.zeros((len(possible_actions),len(states)))
    
    current_policy = copy.deepcopy(policy)
    
    for _ in range(episodes): # Loop over the episodes
        ## sample an intial state at random but make sure it is not goal
        s = nr.choice(states, size = 1)[0]
        while(s == goal): s = nr.choice(states, size = 1)[0]
        ## Now choose action following policy
        a_index, a = select_a_prime(s, current_policy, action_index, True, goal)
        
        s_prime = n_states + 1 # Dummy value of s_prime to start loop
        while(s_prime != goal): # Episode ends where get to terminal state   
            ## Get s_prime given s and a
            s_prime = neighbors[s][a]
            
            ## Find the index or indices of maximum action values for s_prime
            ## Break any tie with multiple max values by random selection
            action_values = Q[:,s_prime]
            a_prime_index = nr.choice(np.where(action_values == max(action_values))[0], size = 1)[0]
            a_prime = possible_actions[a_prime_index]
            
            ## Lookup the reward 
            reward = rewards[s][a]
            
            ## Update the action values
            Q[a_index,s] = Q[a_index,s] + alpha * (reward + gamma * Q[a_prime_index,s_prime] - Q[a_index,s])
            
            ## Set action and state for next iteration
            a = a_prime
            a_index = a_prime_index
            s = s_prime

    return(Q)

Q = Q_learning_0(policy, neighbors, rewards, 1000, goal = 0)

for i in range(4):
    print(np.round(Q[i,:].reshape((4,4)), 4))

### Double Q-Learning

The Q-learning algorithm just presented has a significant **bias**. To understand why this might be consider the following thought experiment. In most cases the sampled action values are inaccurate. Some action values will have a positive error and some will have a negative error. In the error is on the order of the values themselves, the $max_a$ operator has a reasonable chance of selecting an action value that is the largest because of this error. However, the $max_a$ operator will never select an action with a low value simply because of the errors. The net result is a bias toward action values with the largest positive error. 

What can be done to correct this situation? One relatively simple and effective algorithm is known as **double Q-learning**. Double Q-learning maintains two tables of action values. The values from one table are used to perform the bootstrap updates of the other table and vice versa. This approach averages out the bias. For two tables, $Q_1$ and $Q_2$ we can express double Q-learning as follows:

$$Q_1(S_t,A_t) = Q_1(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q_2(S_{t+1},argmax_a Q_1(S_{t+1}, a) - Q_1(S_t,A_t) \big] \\
Q_2(S_t,A_t) = Q_2(S_t,A_t) + \alpha \big[ R_{t+1} + \gamma Q_1(S_{t+1},argmax_a Q_2(S_{t+1}, a) - Q_2(S_t,A_t) \big]$$  

With a 0.5 probability one or the other of these expressions is used for the TD update at each time step. While double Q-learning requires twice as much memory, to maintain the two tables, the computational complexity is the same when compared to Q-learning. 

> **Note:** Another **unbiased** one step off policy TD algorithm is known as **expected SARSA**. See Section 6.6 of Sutton and Barto, second edition, for details.   

### Example of Double Q-Learning

The code in the cell below implements the double Q-learning algorithm. The steps are essentially the same as the foregoing code, except for the updates of the two Q tables. Additional details on this algorithm can be seen by reading the code comments.  

Execute this code and examine the results. 

In [ ]:
def double_Q_learning_0(policy, neighbors, rewards, episodes, goal, alpha = 0.2, gamma = 0.9):
    """
    Function to perform SARSA(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    
    ## Initialize possible actions and the action values
    possible_actions = list(rewards[0].keys())
    action_index = list(range(len(list(policy[0].keys()))))
    Q1 = np.zeros((len(possible_actions),len(states)))
    Q2 = np.zeros((len(possible_actions),len(states)))
    
    current_policy = copy.deepcopy(policy)
    
    for _ in range(episodes): # Loop over the episodes
        ## sample an intial state at random but make sure it is not goal
        s = nr.choice(states, size = 1)[0]
        while(s == goal): s = nr.choice(states, size = 1)[0]
        ## Now choose action following policy
        a_index, a = select_a_prime(s, current_policy, action_index, True, goal)
        
        s_prime = n_states + 1 # Dummy value of s_prime to start loop
        while(s_prime != goal): # Episode ends where get to terminal state   
            ## Get s_prime given s and a
            s_prime = neighbors[s][a]
            
            ## Update one or the other action values at random
            if(nr.uniform() <= 0.5):
                ## Find the index or indices of maximum action values for s_prime
                ## Break any tie with multiple max values by random selection
                action_values = Q1[:,s_prime]
                a_prime_index = nr.choice(np.where(action_values == max(action_values))[0], size = 1)[0]
                a_prime = possible_actions[a_prime_index]
                ## Lookup the reward 
                reward = rewards[s][a]
                ## Update Q1 
                Q1[a_index,s] = Q1[a_index,s] + alpha * (reward + gamma * Q2[a_prime_index,s_prime] - Q1[a_index,s])
            
                ## Set action and state for next iteration
                a = a_prime
                a_index = a_prime_index
                s = s_prime
            
            else:
                ## Find the index or indices of maximum action values for s_prime
                ## Break any tie with multiple max values by random selection
                action_values = Q2[:,s_prime]
                a_prime_index = nr.choice(np.where(action_values == max(action_values))[0], size = 1)[0]
                a_prime = possible_actions[a_prime_index]
                ## Lookup the reward 
                reward = rewards[s][a]
                ## Update Q2
                Q2[a_index,s] = Q2[a_index,s] + alpha * (reward + gamma * Q1[a_prime_index,s_prime] - Q2[a_index,s])
            
                ## Set action and state for next iteration
                a = a_prime
                a_index = a_prime_index
                s = s_prime

    return(Q1)

Q = double_Q_learning_0(policy, neighbors, rewards, 2000, goal = 0)

for i in range(4):
    print(np.round(Q[i,:].reshape((4,4)), 4))

### GPI with Double Q Learning

The code in the cell below applies general policy iteration using double Q-learning(0) to estimate the action values.  Additional details on this algorithm can be seen by reading the code comments.  

Execute this code and examine the results. 

In [ ]:
def double_Q_learning_0_GPI(policy, neighbors, reward, cycles, episodes, goal, alpha = 0.2, gamma = 0.9, epsilon = 0.1):
    ## iterate over GPI cycles
    current_policy = copy.deepcopy(policy)
    for _ in range(cycles):
        ## Evaluate policy with SARSA
        Q = double_Q_learning_0(policy, neighbors, rewards, episodes = episodes, goal = goal)
        
        for s in list(current_policy.keys()): # iterate over all states
            ## Find the index action with the largest Q values 
            ## May be more than one. 
            max_index = np.where(Q[:,s] == max(Q[:,s]))[0]
            
            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(Q.shape[0])
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder   
                    
    return(current_policy)                    
 

Double_Q_0_Policy = double_Q_learning_0_GPI(policy, neighbors, rewards, cycles = 10, episodes = 500, goal = 0, alpha = 0.2, epsilon = 0.01)
Double_Q_0_Policy 

In [ ]:
np.round(np.array(td_0_state_values(Double_Q_0_Policy, n_samps = 10000, goal = 0)).reshape((4,4)), 4)

## N-Step Time Differencing

In the previous lesson we examined Monte Carlo RL which requires that each episode reach a terminal state before a value or action value estimate can be made. In the first part of this lesson we have focused on one-step time differencing, TD, algorithms which update estimates at each time step. Now, we will look at algorithms between these two extreme cases, n-step time differencing algorithms. 

We can generalize the concept of the one-step bootstrapping of the return starting with Td(0):

$$G_{t:t+1} = R_{t+1} + \gamma V_{t}(S_{t+1})$$

Here, $V_{t}(S_{t+1})$ is being bootstrapped. 

We can extend this formulation to two-step TD as follows:

$$G_{t:t+2} = R_{t+1} + \gamma R_{t+2} + \gamma^2 V_{t+1}(S_{t+2})$$

In the above it is $V_{t+1}(S_{t+2})$ being bootstrapped. 

For n-step TD the return is computed by the following bootstrap formulation. 

$$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \ldots + \gamma^{n-1} R_{t+n} + \gamma^2 V_{t+n-1}(S_{t+n})$$

Regardless of home many steps are used to compute the return, the value update is:

$$V_{t+n}(S_t) = V_{t+n-1}(S_t) + \alpha \big[ G_{t:t+n} - V_{t+n-1}(S_t) \big],\ 0 \leq t < T]$$

Where $\delta_t = G_{t:t+n} - V_{t+n-1}(S_t) = $ the n-step TD error.

The resulting backup diagrams for general TD(n) is shown in the diagram below. 

<img src="img/TDN.JPG" alt="Drawing" style="width:400px; height:350px"/>
<center> **Backup Diagram for n-step TD** </center>

From Left to right you can see how the backup diagram adds more states and actions. On the extreme right the backup diagram for the MC algorithm is shown. Notice that MC RL does not bootstrap, but samples until the terminal state is reached. 

N-step TD methods are generally considered to have two advantages over single-step methods:
1. N-step methods generally have better conference properties. 
2. N-step methods break the link between time between actions and time when bootstraping occurs.   

### Example of TD(n) for Policy Evaluation

The code in the cell below implements TD(n) policy evaluation. As you can see, the bookkeeping is a bit involved. We must account for the cases where fewer than n steps have been executed at the start of an episode or when the terminal state has been reached. You can find further details of the algorithm by reading the code comments. 

Execute this code for n=4 and examine the results.

In [ ]:
def TD_n(policy, episodes, n, goal, alpha = 0.2, gamma = 0.9, epsilon = 0.1):
    """
    Function to perform TD(N) policy evaluation.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    
    ## Initialize possible actions and the action values
    action_index = list(range(len(list(policy[0].keys()))))
    v = [0]*len(list(policy.keys()))
    
    current_policy = copy.deepcopy(policy)
    
    
    ## sample an initial state at random and make sure is not terminal state
    s = nr.choice(states, size = 1)[0]
    while(s == goal):
        s = nr.choice(states, size = 1)[0]  
        
    for _ in range(episodes): # Loop over the episodes
        T = float("inf")
        tau = 0
        reward_list = []
        t = 0
        
        while(tau != T - 1): # Episode ends where get to terminal state 
            if(t < T):
                ## Choose action given policy
                probs = list(policy[s].values())
                a = list(policy[s].keys())[nr.choice(action_index, size = 1, p = probs)[0]]
                ## The next state given the action
                s_prime, reward = state_values(s, a)
                reward_list.append(reward)  # append the reward to the list
                if(s_prime == goal): T = t + 1  # We reached the terminal state
                
            tau = t - n + 1 ## update the time step being updated

            if(tau >= 0): # Check if enough time steps to compute return
                ## Compute the return
                ## The formula for the first index in the loop is different from Sutton and Barto
                ## but seems to be correct at least for Python.
                G = 0.0 
                for i in range(tau, min(tau + n - 1, T)):
                    G = G + gamma**(i-tau) * reward_list[i]   
                ## Deal with case of where we are not at the terminal state
                if(tau + n < T): G = G + gamma**n * v[s_prime]
                ## Update v
                v[s] = v[s] + alpha * (G - v[s])
            
            ## Set state for next iteration
            if(s_prime != goal):
                s = s_prime
            t = t +1
    return(v)

np.round(np.array(TD_n(policy, episodes = 1000, n = 4, goal = 0, alpha = 0.2, gamma = 0.9)).reshape((4,4)), 4)

## N-step SARSA for control

Just as we used SARSA(0) for the control or policy improvement, we can generalize this algorithm to the n-step case. This is the SARSA(n) algorithm.  

The n-step return is computed by the following bootstrap formulation. 

$$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \ldots + \gamma^{n-1} R_{t+n} + \gamma^2 Q_{t+n-1}(S_{t+n}, A_{t+n}),\ n \geq 1, 0 \leq t < T-n$$

Using this return, the action value update becomes:

$$Q_{t+n}(S_t, A_t) = Q_{t+n-1}(S_t, A_t) + \alpha \big[ G_{t:t+n} - Q_{t+n-1}(S_t,A_t) \big],\ 0 \leq t < T]$$

Where $\delta_t =  G_{t:t+n} - Q_{t+n-1}(S_t,A_t) = $ the n-step error.

Backup diagrams for n-step SARSA are shown in the figure below. These backups reverse the order of state and action when compared to the n-step TD prediction algorithm discussed above. 

<img src="img/SARSAN.JPG" alt="Drawing" style="width:400px; height:350px"/>
<center> **Backup Diagram for n-step SARSA** </center>

### Example of N-Step SARSA

The code in the cell below implements the n-step SARSA algorithm. As is the case for the n-step TD algorithm, the bookkeeping is a bit involved. We must account for the cases where fewer than n steps have been executed at the start of an episode or when the terminal state has been reached. You can find further details of the algorithm by reading the code comments.

Execute this code for n=4 and examine the results.

In [ ]:
def SARSA_n(policy, episodes, n, goal, alpha = 0.1, gamma = 0.9, epsilon = 0.1):
    """
    Function to perform SARSA(N) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    
    ## Initialize possible actions and the action values
    action_index = list(range(len(list(policy[0].keys()))))
    Q = np.zeros((len(action_index),len(states)))
    
    current_policy = copy.deepcopy(policy)
    
    for _ in range(episodes): # Loop over the episodes
        ## sample a state at random and make sure is not terminal state
        s = nr.choice(states, size = 1)[0]
        while(s == goal):
            s = nr.choice(states, size = 1)[0]
        ## Now choose action given policy
        a_index, a = select_a_prime(s, current_policy, action_index, True, goal)
        
        t = 0 # Initialize the time step count
        T = float("inf")
        tau = 0
        reward_list = []
        while(tau != T - 1): # Episode ends where get to terminal state 
            if(t < T):
                ## The next state given the action
                s_prime, reward = state_values(s, a)
                reward_list.append(reward)  # append the reward to the list
                if(s_prime == goal): T = t + 1  # We reached the terminal state
                else:
                    # Select and store the next action using the policy
                    a_prime_index, a_prime = select_a_prime(s_prime, current_policy, action_index, True, goal)
                
                
            tau = t - n + 1 ## update the time step being updated
  
            if(tau >= 0): # Check if enough time steps to compute return
                ## Compute the return
                ## The formula for the first index in the loop is different from Sutton and Barto
                ## but seems to be correct at least for Python.
                G = 0.0 
                for i in range(tau, min(tau + n, T)):
                    G = G + gamma**(i-tau) * reward_list[i]   
                ## Deal with case of where we are not at the terminal state
                if(tau + n < T): G = G + gamma**n * Q[a_prime_index,s_prime]
                ## Finally, update Q
                Q[a_index,s] = Q[a_index,s] + alpha * (G - Q[a_index,s])
            
            ## Set action and state for next iteration
            if(s_prime != goal):
                s = s_prime   
                a = a_prime 
                a_index = a_prime_index
                
            
            ## increment t
            t = t + 1
    return(Q)

Q = SARSA_n(policy, episodes = 1000, n = 4, goal = 0, alpha = 0.2, gamma = 0.9)

for i in range(4):
    print(np.round(Q[i,:].reshape((4,4)), 4))

### GPI with N-step SARSA

The code in the cell below executes the GPI algorithm using n-step SARSA to evaluate the action values. Details of the algorithm can be found by reading the code comments. 

Execute the code using 50 cycles of 100 episodes and 4-step SARSA and examine the results. 

In [ ]:
def SARSA_n_GPI(policy, n, cycles, episodes, goal, alpha = 0.2, gamma = 0.9, epsilon = 0.1):
    ## iterate over GPI cycles
    current_policy = copy.deepcopy(policy)
    for _ in range(cycles):
        ## Evaluate policy with SARSA
        Q = SARSA_n(policy, episodes, n, goal = goal, alpha = alpha, gamma = gamma, epsilon = epsilon)
        
        for s in list(current_policy.keys()): # iterate over all states
            ## Find the index action with the largest Q values 
            ## May be more than one. 
            max_index = np.where(Q[:,s] == max(Q[:,s]))[0]
            
            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(Q.shape[0])
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder   
                    
    return(current_policy)                    
 

SARSA_N_Policy = SARSA_n_GPI(policy, n = 4, cycles = 50, episodes = 100, goal = 0, alpha = 0.2, epsilon = 0.1)
SARSA_N_Policy

Now, execute the code in the cell below to evaluate the policy you have just computed using the n-step TD algorithm with n = 4. 

In [ ]:
np.array(TD_n(SARSA_N_Policy, episodes = 1000, n = 4, goal = 0, alpha = 0.2, gamma = 0.9)).reshape((4,4))

## N-Step Off-Policy Learning with Importance Sampling

For n-step off-policy learning we update a target policy $\pi(A_t|S_t)$ using samples from a behavior policy $b(A_t|S_t)$. Since the two policies differ, the probabilities of an action given the state will undoubtedly differ. For example, the behavior policy can be exploratory whereas, the target policy is greedy. 

To account for the different probabilities of sampling we reweight by the **importance sampling ratio**. For an n-step algorithm at time step $t$ the importance sampling ratio can be expressed as:

$$\rho_{t:t + n -1} = \prod_{k=\tau}^{min(t + n -1,T-1)} \frac{\pi(A_k|S_k)}{b(A_k|S_k)}$$

The n-step TD update then becomes:

$$V_{t+n}(S_t) = V_{t+n-1}(S_t) + \alpha\ \rho_{t:t+n-1} \big[ G_{t:t+n} - V_{t+n-1}(S_t) \big],\ 0 \leq t < T]$$

And the SARSA update becomes:

$$Q_{t+n}(S_t, A_t) = Q_{t+n-1}(S_t, A_t) + \alpha\ \rho_{t:t+n-1} \big[ G_{t:t+n} - Q_{t+n-1}(S_t,A_t) \big],\ 0 \leq t < T]$$

For both of the above update equations consider the effect of importance sampling ratio. If the action given state is more likely under the target policy that the behavior policy, more weight is given to updating with the error term. However, If the action given state is less likely under the target policy that the behavior policy, less weight is given to updating with the error term. In this way, the weighting by the importance sampling ratio gives the correct updates for the target policy regardless of the transition probabilities of the behavior policy. 

#### Copyright 2018, Stephen F Elston. All rights reserved. 